# Tarea 5

**Envío del PR inicial:** 15 de octubre

**Aceptación del PR:** 21 de octubre

NOTA: Esta tarea debe entregarse en equipo.

## 1

Llamemos $c_n$ el valor del parámetro $c$ donde ocurre la bifurcación de doblamiento de periodo para el mapeo $Q_c(x)=x^2+c$, donde la órbita de periodo $2^n$ nace. Como hemos visto en notebooks anteriores, tenemos que $c_0=1/4$ marca la aparición del atractor de periodo $2^0=1$, $c_1=-1/4$ corresponde a la aparición del atractor de periodo $2^1=2$, $c_2=-3/4$ a la aparición del atractor de periodo $2^2=4$, etc. 

A partir de estos valores y otros que calcularán (al menos deben encontrar $c_6$), definimos la secuencia: $\{f_0, f_1, f_2, \dots\}$, donde

\begin{equation}
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
\end{equation}

La pregunta es, ¿a qué valor converge esta secuencia?, es decir, dar una estimación de $f_\infty$.


*Hint:* Para realizar este ejercicio deben calcular el atractor para varios valores de $c$, de tal manera que puedan aislar las órbitas de periodo $2^p$ y de ahí determinar varios valores $c_n$. Se requerir suficiente cuidado para obtener una buena aproximación de $c_n$. 
Una opción se basa en recordar/usar que las bifurcaciones de doblamiento de periodo ocurren cuando los puntos de la órbita de periodo $p$ se tornan en repulsores, es decir, $(Q_c^p)'(x)=-1$. Esta opción, entonces, involucra obtener los valores $c_n$ a partir de órbitas periódicas de periodo $2^n$ usando los polinomios $Q_c^{2^p}(x)$ y diferenciación automática.

In [16]:
##f_n devuelve la distancia entre el valor c_n (donde ocurre la n-esima bifurcacion) y el de valor c_n+1 

function f_n(cn, cn1, cn2)
    
    f_n = (cn - cn1) / (cn1 - cn2)
    
end

f_n (generic function with 1 method)

In [295]:
iterate(x->x, 2)

LoadError: [91mUndefVarError: iterate not defined[39m

In [347]:
function compone(f0::Function, n_comp)
    
    fn(x) = f0(x)
    f_2n(x) = fn(f0(x))
    f_3n(x) = f_2n(fn(x))
    f_4n(x) = f_3n(f0(x))
    f_5n(x) = f_4n(f0(x))
    f_6n(x) = f_5n(f0(x))
    f_7n(x) = f_6n(f0(x))
    f_8n(x) = f_7n(f0(x))
    f_9n(x) = f_8n(f0(x))
    f_10n(x) = f_9n(f0(x))
    
    if n_comp==0
    return fn
        
        elseif n_comp==1
        return f_2n
        
    elseif n_comp==2
        return f_3n
        
    elseif n_comp==3
        return f_4n
        
    elseif n_comp==4
        return f_5n
        
    elseif n_comp==5
        return f_6n
        
    elseif n_comp==6
        return f_7n
        
    elseif n_comp==7
        return f_8n
        
    elseif n_comp==8
        return f_9n
        
    elseif n_comp==9
        return f_10n
        
    end
end
    
    #for i in 1:n_comp
       # fnnn(x) = fnn(fn(f0(x)))
        #fnn(x) = fnnn(x) 
    #end
    #return fnnn
#end
    

compone (generic function with 1 method)

In [348]:
f(x) = x^2

f (generic function with 1 method)

In [349]:
octava = compone(x -> x+1, 4)

(::f_5n) (generic function with 1 method)

In [350]:
octava(1)

6

## 2

Repitan el ejercicio anterior para el mapeo $S_c(x) = c \sin(x)$. 

- ¿Cómo se comparan los valores obtenidos de $f_n$? 

- ¿Qué interpretación le pueden dar a este resultado, en comparación del ejercicio anterior?

## 3

Como se ve en la Fig. 1 del diagrama de bifurcaciones de $Q_c$, $x=0$ pertenece a un ciclo de periodo $2^n$ para ciertos valores $C_n$ del parámetro. Dichos valores son *especiales*, ya que el hecho de que $x=0$ pertenezca a un ciclo de periodo $2^n$ define los llamados *ciclos superestable*, donde tenemos $(Q^{2^p}_{C_n})'(0)=0$.

- ¿A qué converge la secuencia $f_n$, definida ahora a partir de los valores $C_n$.

- De los $2^p$ puntos del ciclo de periodo $2^p$, es decir, $\{0, p_1, \dots p_{2^{n-1}}\,\}$ hay uno (distinto del 0) cuya distancia a 0 es la menor; a esa distancia la identificaremos como $d_n$. Calcular numéricamente a qué converge la secuencia $d_n/d_{n+1}$.